In [79]:
import pandas as pd
import copy

In [80]:
# Load the training dataset
file_path_train = "/Users/huaijinsun/Downloads/training.xlsx"
df_train = pd.read_excel(file_path_train)

# Load the testing dataset
file_path_test = "/Users/huaijinsun/Downloads/scoring.xlsx"
df_test = pd.read_excel(file_path_test)

In [81]:
# Tidy data

# Remove the 'Region' column
df_train = df_train.drop(columns=['Region'])
df_test = df_test.drop(columns=['Region'])


In [82]:
# Convert all values in 'GVWR Class' column to strings
df_train['GVWR Class'] = df_train['GVWR Class'].astype(str)
df_test['GVWR Class'] = df_test['GVWR Class'].astype(str)

In [83]:
from sklearn.preprocessing import LabelEncoder

# Convert all values in 'GVWR Class' column to strings
df_train['GVWR Class'] = df_train['GVWR Class'].astype(str)
df_test['GVWR Class'] = df_test['GVWR Class'].astype(str)
                                                        
df_train['Number of Vehicles Registered at the Same Address'] = df_train['Number of Vehicles Registered at the Same Address'].astype(str)
df_test['Number of Vehicles Registered at the Same Address'] = df_test['Number of Vehicles Registered at the Same Address'].astype(str)

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Vehicle Category' column
df_train['Vehicle Category'] = label_encoder.fit_transform(df_train['Vehicle Category'])
df_test['Vehicle Category'] = label_encoder.transform(df_test['Vehicle Category'])

# Fit and transform the 'GVWR Class' column
df_train['GVWR Class'] = label_encoder.fit_transform(df_train['GVWR Class'])
df_test['GVWR Class'] = label_encoder.transform(df_test['GVWR Class'])

# Fit and transform the 'Fuel Type' column
df_train['Fuel Type'] = label_encoder.fit_transform(df_train['Fuel Type'])
df_test['Fuel Type'] = label_encoder.transform(df_test['Fuel Type'])

# Fit and transform the 'Future Technology' column
df_train['Fuel Technology'] = label_encoder.fit_transform(df_train['Fuel Technology'])
df_test['Fuel Technology'] = label_encoder.transform(df_test['Fuel Technology'])

# Fit and transform the 'Electric Mile Range' column
df_train['Electric Mile Range'] = label_encoder.fit_transform(df_train['Electric Mile Range'])
df_test['Electric Mile Range'] = label_encoder.transform(df_test['Electric Mile Range'])

# Apply the custom function to the 'Number of Vehicles Registered at the Same Address' column

df_train['Number of Vehicles Registered at the Same Address'] = label_encoder.fit_transform(df_train['Number of Vehicles Registered at the Same Address'])
df_test['Number of Vehicles Registered at the Same Address'] = label_encoder.fit_transform(df_test['Number of Vehicles Registered at the Same Address'])

# Display the first few rows of the updated dataset
print(df_train.head())
print(df_test.head())

   Date  Vehicle Category  GVWR Class  Fuel Type  Model Year  Fuel Technology  \
0  2019                 5           8          2      2020.0                2   
1  2020                 5           8          2      2020.0                2   
2  2021                 5           8          2      2020.0                2   
3  2019                 5           8          2      2019.0                2   
4  2019                 5           8          2      2018.0                2   

   Electric Mile Range  Number of Vehicles Registered at the Same Address  \
0                    4                                                  4   
1                    4                                                  0   
2                    4                                                  0   
3                    4                                                  4   
4                    4                                                  4   

   Vehicle Population  
0              395883  
1 

In [84]:
# Fill the model year empty entry with the mean
df_train['Model Year'] = df_train['Model Year'].fillna(df['Model Year'].mean())
df_test['Model Year'] = df_test['Model Year'].fillna(df['Model Year'].mean())

In [85]:
# Convert model year to normal distribution
df_train['Model Year'] = df_train['Model Year'].apply(lambda x: (x - df['Model Year'].mean()) / df['Model Year'].std())

# Convert date to normal distribution
df_train['Date'] = df_train['Date'].apply(lambda x: (x - df['Date'].mean()) / df['Date'].std())

# Convert model year to normal distribution
df_test['Model Year'] = df_test['Model Year'].apply(lambda x: (x - df['Model Year'].mean()) / df['Model Year'].std())

# Convert date to normal distribution
df_test['Date'] = df_test['Date'].apply(lambda x: (x - df['Date'].mean()) / df['Date'].std())

print(df_train.head())
print(df_test.head())

          Date  Vehicle Category  GVWR Class  Fuel Type   Model Year  \
0  2018.975410                 5           8          2  2019.975398   
1  2019.975398                 5           8          2  2019.975398   
2  2020.975385                 5           8          2  2019.975398   
3  2018.975410                 5           8          2  2018.975410   
4  2018.975410                 5           8          2  2017.975422   

   Fuel Technology  Electric Mile Range  \
0                2                    4   
1                2                    4   
2                2                    4   
3                2                    4   
4                2                    4   

   Number of Vehicles Registered at the Same Address  Vehicle Population  
0                                                  4              395883  
1                                                  0              370954  
2                                                  0              349406  
3       

In [86]:
# Handle missing values (example: fill with mean)
# df_train = df_train.fillna(df_train.mean())
df_test = df_test.fillna(df_test.mean())

In [87]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

# Normalize numerical features
scaler = StandardScaler()
numerical_features_train = df_train.select_dtypes(include=['float64', 'int64']).columns
numerical_features_test = df_test.select_dtypes(include=['float64', 'int64']).columns
df_train[numerical_features_train] = scaler.fit_transform(df_train[numerical_features_train])
df_test[numerical_features_test] = scaler.transform(df_test[numerical_features_test])

In [88]:
# Define the target and features
X_train = df_train.drop(columns=['Vehicle Population']) 
y_train = df_train['Vehicle Population']  
X_test = df_test.drop(columns=['Vehicle Population'])  
y_test = df_test['Vehicle Population']  

# Define the K-fold Cross Validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store results
mse_scores = []

# K-fold Cross Validation model evaluation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Build the neural network model
    model = Sequential()
    model.add(Dense(64, input_dim=X_train_fold.shape[1], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    
    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    # Train the model
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, verbose=0)

    # Evaluate the model on the validation fold
    y_val_pred = model.predict(X_val_fold)
    mse = mean_squared_error(y_val_fold, y_val_pred)
    mse_scores.append(mse)

# Calculate the average MSE from cross-validation
average_mse_cv = np.mean(mse_scores)
print(f"Cross-Validation Mean Squared Error: {average_mse_cv}")

# Evaluate the model on the testing dataset
y_test_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
print(f"Testing Mean Squared Error: {mse_test}")

# Testing Mean Squared Error: 0.26270263641985175

257/257 [==============================] - 0s 227us/step


257/257 [==============================] - 0s 199us/step


257/257 [==============================] - 0s 204us/step


257/257 [==============================] - 0s 203us/step


257/257 [==============================] - 0s 192us/step
Cross-Validation Mean Squared Error: 0.18293351160156307
236/236 [==============================] - 0s 184us/step
Testing Mean Squared Error: 0.23819876551494648


In [89]:
# Implement a neural network model to it to predict the Vehicle Population
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

# Define the target and features
X_train = df_train.drop(columns=['Vehicle Population']) 
y_train = df_train['Vehicle Population']  
X_test = df_test.drop(columns=['Vehicle Population'])  
y_test = df_test['Vehicle Population']  


# Create a pipeline with standardization and MLPRegressor
pipeline = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42))
# Fit the model
pipeline.fit(X_train, y_train)
# Make predictions
y_pred = pipeline.predict(X_test)
# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

In [90]:
print("Mean Squared Error:", mse)

Mean Squared Error: 0.3220264745127927


In [91]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the target and features
X_train = df_train.drop(columns=['Vehicle Population']) 
y_train = df_train['Vehicle Population']  
X_test = df_test.drop(columns=['Vehicle Population'])  
y_test = df_test['Vehicle Population']  

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Epoch 1/100
1027/1027 [==============================] - 1s 377us/step - loss: 1.1454 - val_loss: 0.0273
Epoch 2/100
1027/1027 [==============================] - 0s 335us/step - loss: 0.7795 - val_loss: 0.0332
Epoch 3/100
1027/1027 [==============================] - 0s 334us/step - loss: 0.4597 - val_loss: 0.0540
Epoch 4/100
1027/1027 [==============================] - 0s 335us/step - loss: 0.3548 - val_loss: 0.0451
Epoch 5/100
1027/1027 [==============================] - 0s 334us/step - loss: 0.3013 - val_loss: 0.0575
Epoch 6/100
1027/1027 [==============================] - 0s 335us/step - loss: 0.2835 - val_loss: 0.0504
Epoch 7/100
1027/1027 [==============================] - 0s 334us/step - loss: 0.2651 - val_loss: 0.0477
Epoch 8/100
1027/1027 [==============================] - 0s 335us/step - loss: 0.2831 - val_loss: 0.0659
Epoch 9/100
1027/1027 [==============================] - 0s 335us/step - loss: 0.2825 - val_loss: 0.0538
Epoch 10/100
1027/1027 [==============================]